In [2]:
import pandas as pd
import time # Used in Metar
import datetime # Used in Metar
from urllib.request import urlopen # Used in Metar
import sqlite3 as sql 
import numpy as np
from multiprocessing import Pool

# Getting METAR Data

In [25]:
df = pd.read_csv('eastern_canada_airports.csv')

In [26]:
df

,stid,station_name,lat,lon,elev,begints,iem_network
0,CWAR,ARGENTIA (AUT),47.29389,-53.99333,16.00000,1945-02-28 00:00,CA_NF_ASOS
1,CWKW,CAPE KAKKIVIAK,59.98500,-64.16500,551.00000,1996-02-21 00:00,CA_NF_ASOS
2,CWYK,CAPE KIGLAPAIT,57.13583,-61.47556,834.00000,1998-12-23 00:00,CA_NF_ASOS
3,CWRA,CAPE RACE (AUT) NFLD,46.66000,-53.07639,28.00000,1955-07-02 00:00,CA_NF_ASOS
4,CYCA,Cartwright,53.68280,-57.04190,-4.33276,1979-01-30 00:00,CA_NF_ASOS
...,...,...,...,...,...,...,...
290,CYOY,VALCARTIER AIRPORT,46.90030,-71.50330,168.00000,1978-10-30 00:00,CA_QC_ASOS
291,CYVO,Val D Or,48.05640,-77.78670,337.00000,1955-07-02 00:00,CA_QC_ASOS
292,CWHM,VARENNES QUE,45.72306,-73.37667,192.00000,1994-08-04 00:00,CA_QC_ASOS
293,CWVY,Villeroy,46.44960,-71.91370,107.00000,2002-08-11 00:00,CA_QC_ASOS


In [28]:
can_stations = df['stid'].tolist()

In [30]:
can_stations[0]

'CWAR'

In [ ]:
hurricane_metars = pd.DataFrame()

In [3]:
def construct_urls(station,start_date,finish_date):
  try: 
    startts = datetime.datetime(*start_date)
    endts = datetime.datetime(*finish_date)

    SERVICE = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"
    MAX_ATTEMPTS = 6 
    service = SERVICE + "data=all&tz=Etc/UTC&format=comma&latlon=yes&"

    service += startts.strftime("year1=%Y&month1=%m&day1=%d&")
    service += endts.strftime("year2=%Y&month2=%m&day2=%d&")
    return [f"{service}&station={station}" for station in station]
  
  except:
    print(f'Bad data with: {station} - using {start_date} and {finish_date}')

In [33]:
url = []
for year in range (2014, 2022):
    start_date = (year,5,1)
    finish_date = (year,11,30)
    url += construct_urls(can_stations, start_date, finish_date)

In [34]:
url

['http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=comma&latlon=yes&year1=2021&month1=05&day1=01&year2=2021&month2=11&day2=30&&station=C',
 'http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=comma&latlon=yes&year1=2021&month1=05&day1=01&year2=2021&month2=11&day2=30&&station=W',
 'http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=comma&latlon=yes&year1=2021&month1=05&day1=01&year2=2021&month2=11&day2=30&&station=A',
 'http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=comma&latlon=yes&year1=2021&month1=05&day1=01&year2=2021&month2=11&day2=30&&station=R']

In [46]:
def pull_metar(url):
  return pd.read_csv(url, skiprows=5)

In [ ]:
hurricane_metar = pd.DataFrame()

In [ ]:
if __name__ == '__main__':
    pool = Pool()

# Create a multiprocessing Pool
    for r in pool.map(pull_metar, url):
      hurricane_metar = pd.concat([hurricane_metar,r], ignore_index=True)

In [ ]:
hurricane_metar.head()

,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,...,wxcodes,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar,snowdepth
0,CWRA,2021-05-25 00:00,-53.0764,46.66,29.5,M,M,330.0,8.0,0.0,...,M,M,M,M,M,M,M,M,CWRA 250000Z AUTO 33008KT M01/ RMK AO1 SLP181 ...,M
1,CWRA,2021-05-25 01:00,-53.0764,46.66,28.6,M,M,320.0,7.0,0.0,...,M,M,M,M,M,M,M,M,CWRA 250100Z AUTO 32007KT M02/ RMK AO1 SLP187 ...,M
2,CWRA,2021-05-25 02:00,-53.0764,46.66,26.8,M,M,300.0,5.0,0.0,...,M,M,M,M,M,M,M,M,CWRA 250200Z AUTO 30005KT M03/ RMK AO1 SLP190 ...,M
3,CWRA,2021-05-25 03:00,-53.0764,46.66,29.5,M,M,250.0,9.0,0.0,...,M,M,M,M,M,M,M,M,CWRA 250300Z AUTO 25009KT M01/ RMK AO1 SLP196 ...,M
4,CWRA,2021-05-25 04:00,-53.0764,46.66,29.7,M,M,240.0,7.0,0.0,...,M,M,M,M,M,M,M,M,CWRA 250400Z AUTO 24007KT M01/ RMK AO1 SLP195 ...,M


In [ ]:
hurricane_metar = hurricane_metar._convert(numeric=True)
hurricane_metar.apply(pd.api.types.infer_dtype)
hurricane_metar['tmpf'] = hurricane_metar.apply(lambda x: (5/9)*(x['tmpf']-32),axis=1)
hurricane_metar['dwpf'] = hurricane_metar.apply(lambda x: (5/9)*(x['dwpf']-32),axis=1)
hurricane_metar['feel'] = hurricane_metar.apply(lambda x: (5/9)*(x['feel']-32),axis=1)
hurricane_metar.rename({'drct': 'wnd_dir', 'sknt': 'wnd_spd', 'tmpf':'temp', 'dwpf':'dew_pt', 'metar': 'raw_metar'}, axis=1, inplace=True)

In [ ]:
cnx = sql.connect('Canadian_Hurricane_Impact.db')
hurricane_metar.to_sql('Metar_Data', cnx, if_exists='append', index = False)

# Filtering Metar

In [ ]:
cnx = sql.connect('Canadian_Hurricane_Impact.db')

In [ ]:
# #Connecting to sqlite
# conn = sql.connect('/content/drive/MyDrive/Canadian_Hurricane_Impact.db')

# #Creating a cursor object using the cursor() method
# cursor = conn.cursor()

# #Doping Metar_Data table if already exists
# cursor.execute("DROP TABLE Metar_Data")
# print("Table dropped... ")

# #Commit your changes in the database
# conn.commit()

# #Closing the connection
# conn.close()

Table dropped... 


In [ ]:
dates = pd.read_sql_query('SELECT ObservationDate FROM Complete_impact', cnx)

In [ ]:
dates['ObservationDate'] = pd.to_datetime(dates['ObservationDate'])

In [ ]:
cnx.close()

In [ ]:
from sqlalchemy import create_engine

In [ ]:
def process_sql_using_pandas(dates):
    engine = create_engine("sqlite:////Canadian_Hurricane_Impact.db")
    conn = engine.connect().execution_options(stream_results=True)
    # dfs = []
    df = pd.DataFrame()
    for chunk_dataframe in pd.read_sql("SELECT * FROM Metar_Data", conn, chunksize=500000):
        # print(f"Got dataframe w/{len(chunk_dataframe)} rows")
        # dfs.append(chunk_dataframe)
        # df = dd.read_sql_table('accounts', 'sqlite:////content/drive/MyDrive/Canadian_Hurricane_Impact.db', npartitions=10, index_col='id')
        chunk_dataframe['valid'] = pd.to_datetime(chunk_dataframe['valid'])
        df = pd.concat([df,chunk_dataframe[chunk_dataframe.valid.dt.strftime('%y%m%d').isin(dates.ObservationDate.dt.strftime('%y%m%d'))]])
         # ... do something with dataframe ...
    conn.close()
    return df

In [ ]:
if __name__ == '__main__':
    filtered_hurricane_metar = pd.DataFrame()
    filtered_hurricane_metar = process_sql_using_pandas(dates)

In [ ]:
filtered_hurricane_metar = pd.read_sql("SELECT * FROM Filtered_Metar_Data", cnx)

In [ ]:
cnx = sql.connect('Canadian_Hurricane_Impact.db')
filtered_hurricane_metar.to_sql('Filtered_Metar_Data', cnx, if_exists='append', index = False)

In [ ]:
cnx.close()

In [ ]:
display(filtered_hurricane_metar)